# ETL 설계 1편: 메달리온 아키텍처

Bronze, Silver, Gold. 데이터를 레이어별로 나눠서 적재하면 뭐가 달라지는가.

DuckDB와 dbt로 직접 구성해 본다.

📖 [블로그 원문](https://biz-agentic-ai.github.io/guides/etl-design/001-medallion-architecture/)

## 환경 세팅

In [ ]:
# 도구 설치 (protobuf 경고는 무시해도 됩니다 - Colab의 tensorflow와 버전 충돌이지만 실습에 영향 없음)
!pip install -q duckdb dbt-core dbt-duckdb 2>&1 | grep -v "protobuf"

In [ ]:
import duckdb

# DuckDB 데이터베이스 생성
conn = duckdb.connect('warehouse.duckdb')
print(f"DuckDB {duckdb.__version__} 준비 완료")

## Bronze 레이어: 원본 그대로 적재

소스 시스템에서 가져온 데이터를 변환 없이 저장한다.

CSV든 JSON이든 API 응답이든 있는 그대로. 타입 캐스팅도 안 하고, 컬럼명도 안 바꾼다.

In [ ]:
conn.execute("""
CREATE SCHEMA IF NOT EXISTS bronze;

CREATE TABLE bronze.orders AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_orders.csv');

CREATE TABLE bronze.customers AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_customers.csv');

CREATE TABLE bronze.payments AS
SELECT * FROM read_csv_auto('https://raw.githubusercontent.com/dbt-labs/jaffle_shop/main/seeds/raw_payments.csv');
""")

# 적재 확인
conn.execute("SELECT count(*) as cnt FROM bronze.orders").fetchdf()

## 적재된 Bronze 데이터 확인

어떤 변환도 하지 않았다. 타입 캐스팅도 안 했고, 컬럼명도 바꾸지 않았다. 이게 Bronze의 원칙이다.

In [ ]:
conn.execute("SELECT * FROM bronze.orders LIMIT 5").fetchdf()

In [ ]:
conn.execute("SELECT * FROM bronze.customers LIMIT 5").fetchdf()

In [ ]:
conn.execute("SELECT * FROM bronze.payments LIMIT 5").fetchdf()

## 레이어 구조 확인

```
소스 시스템 → [Bronze] → [Silver] → [Gold] → BI / 분석
               원본 적재    정제·표준화   비즈니스 집계
```

Bronze에 적재했다. 여기서 바로 분석 쿼리를 던지고 싶은 유혹이 생긴다.

참아야 한다. Bronze 데이터를 직접 분석에 쓰면 3개월 뒤에 데이터 늪에 빠진다.

다음 노트북에서 Bronze를 Silver로 올리는 과정을 다룬다.

In [ ]:
# 각 테이블 건수 확인
for table in ['orders', 'customers', 'payments']:
    cnt = conn.execute(f"SELECT count(*) FROM bronze.{table}").fetchone()[0]
    print(f"bronze.{table}: {cnt}건")

In [ ]:
# 정리
conn.close()